In [5]:

import torch
from torch.utils.data import Dataset, DataLoader
import scipy.io as sio
from pathlib import Path


In [2]:
! python --version

Python 3.12.3


In [3]:
torch.__version__

'2.9.0+cu128'

In [4]:
torch.cuda.is_available()

True

# Rutas

In [6]:
BASE_DIR = Path().resolve().parents[0]
DATA_PROC_DIR = BASE_DIR / "data" / "processed"

# Carga de datos

In [10]:
path_data = DATA_PROC_DIR / "weather_data_processed.mat"
WS_data = sio.loadmat(path_data)

In [ ]:
# Malla PINN
X_PINN = WS_data["X_PINN"]
Y_PINN = WS_data["Y_PINN"]
T_PINN = WS_data["T_PINN"]
# Data WS
T_WS = WS_data["T_WS"]
P_WS = WS_data["P_WS"]
U_WS = WS_data["U_WS"]
V_WS = WS_data["V_WS"]
X_WS = WS_data["X_WS"]
Y_WS = WS_data["Y_WS"]
# Data val
WS_val = WS_data["WS_val"]
T_val = WS_data["T_val"]
P_val = WS_data["P_val"]
U_val = WS_data["U_val"]
V_val = WS_data["V_val"]
X_val = WS_data["X_val"]
Y_val = WS_data["Y_val"]
Z_val = WS_data["Z_val"]

L = WS_data["L"]
W = WS_data["W"]
P0 = WS_data["P0"]
Re = WS_data["Re"]

In [13]:
X_PINN.shape

(160, 2016)

# train, test, split